## Import libraries

In [11]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap
from fiona.crs import from_epsg
import matplotlib as mpl
import matplotlib.pyplot as plt
import bokeh
from bokeh import plotting
from bokeh.models import GeoJSONDataSource, HoverTool
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.tile_providers import CARTODBPOSITRON
import psycopg2

NoneType = type(None)

%matplotlib inline

## Data addresses

1. **Stations request** http://api.gios.gov.pl/pjp-api/rest/station/findAll
2. **Sensors request** http://api.gios.gov.pl/pjp-api/rest/station/sensors/{stationId}
3. **Data request** http://api.gios.gov.pl/pjp-api/rest/data/getData/{sensorId}
4. **AQ index request** http://api.gios.gov.pl/pjp-api/rest/aqindex/getIndex/{stationId}

## Get the data

In [2]:
stations = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/findAll").json()
print("Number of available stations: ", len(stations))

Number of available stations:  157


In [3]:
stations_dict = {}
station_ids = []
station_lats = []
station_lons = []
station_geometries = []

for station in stations:
    
    station_ids.append(station["id"])
    #station_lats.append(float(station["gegrLat"]))
    #station_lons.append(float(station["gegrLon"]))
    station_geometries.append(Point(float(station["gegrLon"]), float(station["gegrLat"])))
    
stations_dict["station_id"] = station_ids
#stations_dict["lat"] = station_lats
#stations_dict["lon"] = station_lons
stations_dict["station_id"] = station_ids
stations_dict["geometry"] = station_geometries

In [4]:
stations_df = gpd.GeoDataFrame(stations_dict)
stations_df.crs = from_epsg(4326)
stations_df.head()

,station_id,geometry
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


## Plot stations on map

In [5]:
stations_map = folium.Map([52, 19], zoom_start=6, tiles='Stamen Terrain')

points = folium.features.GeoJson(stations_df.to_json())

stations_map.add_child(points)
#stations_map.add_child(HeatMap([[row["lat"], row["lon"]] for name, row in stations_df.iterrows()]))
stations_map

## Check available readings for stations

In [6]:
sensors_dict = {}
stations_ids = []
sensors_ids = []
sensors_param = []

for station in stations:
    
    station_id = station["id"]
    sensors = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/sensors/{}".format(station_id)).json()
    
    for sensor in sensors:

        stations_ids.append(sensor["stationId"])
        sensors_ids.append(sensor["id"])
        sensors_param.append(sensor["param"]["paramCode"])
    
sensors_dict["station_id"] = stations_ids
sensors_dict["sensor_id"] = sensors_ids
sensors_dict["parameter"] = sensors_param\

In [9]:
sensors_df = pd.DataFrame(sensors_dict)
sensors_df.head()

,station_id,sensor_id,parameter
0,114,642,NO2
1,114,644,O3
2,117,660,CO
3,117,14395,PM10
4,117,658,C6H6


## Check data

In [10]:
def get_sensor_readings(row):
    
    sensor_id = row["sensor_id"]
    data_json = requests.get("http://api.gios.gov.pl/pjp-api/rest/data/getData/{}".format(sensor_id)).json()
    count = 0
    try:
        data = data_json["values"][count]["value"]
        while isinstance(data, NoneType):
            data = data_json["values"][count]["value"]
            count+=1
        return data
    except:
        return np.NaN

In [11]:
sensors_df['data'] = sensors_df.apply(get_sensor_readings, axis=1)
sensors_df.head()

,station_id,sensor_id,parameter,data
0,114,642,NO2,12.76580
1,114,644,O3,99.66110
2,117,660,CO,381.52700
3,117,14395,PM10,19.23480
4,117,658,C6H6,0.00035


In [12]:
available_parameters = list(sensors_df.parameter.unique())
available_parameters

['NO2', 'O3', 'CO', 'PM10', 'C6H6', 'PM2.5', 'SO2']

In [13]:
def get_param_df(parameter=''):
    
    param_df = sensors_df[sensors_df["parameter"]=="{}".format(parameter)]
    param_df = gpd.GeoDataFrame(pd.merge(param_df, stations_df, on='station_id'))
    param_df.dropna(inplace=True)
    param_df.crs = from_epsg(4326)
    
    return param_df

In [14]:
def show_readings(parameter=''):
    
    param_df = get_param_df("{}".format(parameter))
    param_df = param_df.to_crs(epsg=3395)  # conversion to World Mercator needed
    
    plotting.output_notebook()

    # reduce differences between readings
    size=param_df["data"]
    # get data resolution
    radii = np.array(param_df['data'].apply(int))
    #create colors list
    colors = ["#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 255*mpl.cm.RdYlGn(1-mpl.colors.Normalize()(radii))]

    p = plotting.figure(toolbar_location="left", 
                        plot_width=900, 
                        plot_height=700, 
                        x_axis_type="mercator", 
                        y_axis_type="mercator")

    p.circle(param_df['geometry'].x, 
             param_df['geometry'].y, 
             size=size, 
             fill_color=colors, 
             fill_alpha=0.8, 
             line_color=None)

    p.add_tile(CARTODBPOSITRON)

    plotting.show(p)

In [20]:
show_readings("O3")

Loading BokehJS ...

## Connect with database

In [40]:
try:
    conn = psycopg2.connect("dbname='haqs' user='postgres' host='localhost' password='postgres'")
except:
    print ("I am unable to connect to the database")
    conn.close()

In [38]:
#conn.close()

### Show Tables

In [ ]:
sql = "SELECT * FROM pg_catalog.pg_tables;"

cur = conn.cursor()
cur.execute(sql)
cur.fetchall()

### Create Station Table

In [49]:
sql = """CREATE TABLE stations ( 
  station_id INTEGER PRIMARY KEY
);

SELECT AddGeometryColumn('stations', 'geom', '4326', 'POINT', 2);"""

In [ ]:
cur = conn.cursor()
cur.execute(sql)
cur.fetchall()

### Insert stations into DataBase

1. Using text 
> <i>**INSERT INTO**</i> stations(station_id, geom) **VALUES**(2, ST_GeomFromText('POINT(-71.060316 48.432044)', EPSG));
2. Using lognitude and latitude
> <i>**INSERT INTO**</i> stations(station_id, geom) **VALUES**(2, ST_SetSRID(ST_MakePoint(lon, lat), EPSG));

In [51]:
stations[0]

{'id': 114,
 'stationName': 'Wrocław - Bartnicza',
 'gegrLat': '51.115933',
 'gegrLon': '17.141125',
 'city': {'id': 1064,
  'name': 'Wrocław',
  'commune': {'communeName': 'Wrocław',
   'districtName': 'Wrocław',
   'provinceName': 'DOLNOŚLĄSKIE'}},
 'addressStreet': 'ul. Bartnicza'}

In [44]:
def db_insert_station(conn, station_id, lon, lat):
    """
    Function inserts stations with its coordinates to Database
    """
    sql = "INSERT INTO public.stations (station_id, geom) VALUES (%s, ST_SetSRID(ST_MakePoint(%s, %s), 4326))"
    with conn.cursor() as cur:
        cur.execute(sql, (station_id, lon, lat))
    conn.commit()

In [45]:
# iterate over stations
for station in stations:
    
    station_id = station["id"]
    station_lon = station['gegrLon']
    station_lat = station['gegrLat']
    
    try:
        db_insert_station(conn=conn, station_id=station_id, lon=station_lon, lat=station_lat)
    except:
        print ("Cannot execute insertion for Station ID: {}".format(station_id))

#### Show Insertions

In [46]:
sql = "SELECT * FROM stations"

cur = conn.cursor()
cur.execute(sql)

cur.fetchall()

[(114, '0101000020E6100000E3A59BC4202431407AC37DE4D68E4940'),
 (117, '0101000020E6100000736891ED7C073140346953758F904940'),
 (129, '0101000020E6100000DA0418963F0331407E8CB96B098B4940'),
 (52, '0101000020E6100000BB809719362E30408A7780272D9A4940'),
 (109, '0101000020E6100000C07B478D094530404F3E3DB665624940'),
 (11, '0101000020E6100000C746205ED79F2E4074B515FBCB744940'),
 (14, '0101000020E6100000F4346090F4E12D400EA0DFF76F7C4940'),
 (16, '0101000020E61000003D2CD49AE6A53040F6798CF2CC5D4940'),
 (38, '0101000020E61000000FB4024356A73040C005D9B27C374940'),
 (67, '0101000020E6100000C9570229B18330400C3F389F3A4A4940'),
 (70, '0101000020E6100000DE3EABCC944A3140E9B81AD995784940'),
 (74, '0101000020E6100000A71FD4450ADD2E403F1D8F19A8A84940'),
 (84, '0101000020E610000070438CD7BC7A2F402C11A8FE415E4940'),
 (132, '0101000020E6100000FAB7CB7EDDD1304036CD3B4ED14B4940'),
 (134, '0101000020E6100000234A7B832F042E406E4E250340934940'),
 (9153, '0101000020E6100000A81C93C5FD872F40280B5F5FEB744940'),
 (10007, '010100